# Sistemas de recomendação

O que vamos fazer?

- Explorar a abordagem de filtros colaborativos
- Criar um dataset para resolver por sistemas de recomendação 
- Implementar uma função de custo e descida de gradiente 
- Formar um modelo de recomendação por filtros colaborativos 
- Realizar previsões de recomendações
- Voltar a formar o modelo incorporando novos valores 
- Recomendar exemplos semelhantes

In [ ]:
# TODO: Usa esta célula para importar todas as livrarias necessárias

import time
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance

np.random.seed(42)

# Criar o dataset sintético

Vamos criar um dataset sintético de novo, mas focado nos sistemas de recomendação, com algumas diferenças perante a regressão linear:
- As variáveis previstas ou independentes *X* (de tamanho (m, n + 1)) que representam as características de cada exemplo, **não são conhecidas** de antemão.
- O vetor $\Theta$ (*Theta*) é 2D (de tamanho ($n_u$ + 1)), agora que representa os pesos ou coeficientes das características para cada utilizador. Novamente, não é conhecido de antemão.
- O vetor *Y* é 2D (de tamanho (m, $n_u$)), agora representa a valorização para cada exemplo de cada utilizador.
- O vetor *Y* irá conter tanto as valorizações “reais” que emitiu cada utilizador para cada película que valorizou, como, no final da formação, uma previsão das suas valorizações para recomendar uma película ou outra.
- *R* será uma matriz “máscara” sobre *Y*, utilizada para indicar que valorizações de *Y* são as reais, emitidas por um utilizador e por tanto são aquelas a ter em conta para formar o modelo, e quais são apenas previsões.

Um exemplo habitual são as recomendações de películas num portal de streaming de vídeo. Neste caso, um dataset teria estas características, p. ex.:
- *m*: Nº de películas.
- *n*: Nº de características de cada película e de coeficientes de cada utilizador.
- $n_u$: Nº de utilizadores do portal.
- $n_rr$ e $n_r$: Percentagem de valorizações de cada película por cada utilizador, e n.º de valorizações total, conhecidos de antemão.
- *X*: Matriz 2D de características de cada película, tamanho (n.º de películas, n.º de características).
- $\Theta$: Matriz 2D de coeficientes de cada utilizador para cada película, tamanho (n.º de utilizadores, n.º de características).
- *Y*: Matriz 2D de valorizações de cada utilizador para cada película, tamanho (n.º de utilizadores, n.º de películas).

Para ter à mão, deixamos esta tabela rápida para consultar o tamanho de cada matriz:
- *X*(m, n + 1)
- $\Theta$( $n_u$ , n + 1)
- *Y*(m, $n_u$)

Para não complicar mais a implementação, neste caso não pré-processaremos os dados, já que deveríamos pré-processar *X* e *Y*, que além disso deveríamos passar de uma escala p. ex. 0-5 estrelas a [0, 1].

Na seguinte célula, seguir as instruções para gerar um dataset com as características necessárias para poder resolver por um filtro colaborativo de sistemas de recomendação:

In [ ]:
# TODO: Criar um dataset com as características necessárias para um sistema de recomendação
# Recordar que pode voltar a esta célula e modificar o tamanho do dataset em qualquer momento

m = 10 # N.º de exemplos
n = 4 # N.º de características de cada exemplo/utilizador
n_u = 3 # N.º de utilizador
n_rr = 0.5 # Percentagem de valorizações conhecidas de antemão

# Criar um X com valores aleatórios e tamanho (m, n)
# Inserir uma coluna de 1s na primeira posição
X_verd = [...]
X_verd = [...]

# Criar um Theta_verd com valores aleatórios e tamanho (n_u, n + 1)
Theta_verd = [...]

# Criar um Y_verd multiplicando X_verd e Theta_verd
Y_verd = [...]

# Criar uma matriz R de zeros com tamanho (m, n_u)
r = [...]
count_r = round(n_rr * r.size) # nº de 1s em R
while count_r:
        # Gerar um int aleatório entre [0, m]
    i = [...]
        # Gerar um int aleatório entre [0, n_u]
    j = [...]
    
    # Mudar esse índice de R a 1. se não se mudou antes, e diminuir em 1 o n.º de 1s em R
    if not r[i,j]:
        r[i,j] = 1.
        count_r -= 1
        
# Contar os valores de R que não sejam 0.
n_r = [...]

# Gerar um Y com apenas as valorizações conhecidas usando R
y = [...]

print('Tamanho de X(m, n+1), Theta(n_u, n+1) e Y(m, n_u) verdadeiros:') 
print(X_verd.shape, Theta_verd.shape, Y_verd.shape)
print('Tamanho de y e R conhecidas:') 
print(y.shape, r.shape)
print('N.º de elementos de R, ou valorizações conhecidas:', n_r)

# Função de custo e descida de gradiente

Vamos implementar a função de custo regularizada e a descida de gradiente regularizado para a otimizar e formar o modelo de ML.

Conceptualmente, quanto à formação do modelo por descida de gradiente, vamos seguir uns passos diferentes aos da regressão linear.

Enquanto que na regressão linear eram conhecidos *Y* e *X*, e podíamos resolver iterativamente para $\Theta$, nesta ocasião *X* nem é conhecida de antemão, já que habitualmente é impossível na prática conhecer ou ter anotadas de antemão todas as características de todos os exemplos ou películas.

Além do mais, enquanto que se temos algumas valorizações por cada utilizador de algumas películas, habitualmente temos uma percentagem sob de todas as valorizações de cada utilizador para cada exemplo, pelo que *Y* é outra matriz variável não completamente conhecida de antemão, sem que a maioria dos seus campos irão estar vazios inicialmente.

O nosso objetivo, neste caso, não será tanto resolver $\Theta$ como resolver *Y* para obter todas as valorizações previstas de cada utilizador para 
cada exemplo.

Por tanto, o algoritmo de formação será:

1. Recompilar os exemplos e as valorizações nas matrizes *X*, $\Theta$ e *Y*.
1. Marcar as valorizações conhecidas na matriz dispersa *R*.
1. Dadas *X* e *Y*, podemos prever $\Theta$.
1. Dadas $\Theta$ e *Y*, podemos prever *X*.
1. Estimar de forma iterativa *X* e $\Theta$ em cada iteração porque a formação converta num custo mínimo.
1. Quando dispusermos de mais valorizações, voltamos a formar o modelo adicionando-as a *Y* e marcando-as em *R*.

Na seguinte célula, seguir as instruções para implementar a função de custo e gradient descent regularizados para um filtro colaborativo, seguindo as seguintes fórmulas:

$$
\min\limits_{\theta^0, ..., \theta^{n_u} \\ x^0, ..., x^{n_m}}J(x^0, ..., x^{n_m}, \theta^0, ..., \theta^{n_u}) = \min\limits_{\theta^0, ..., \theta^{n_u} \\ x^0, ..., x^{n_m}} (\frac{1}{2} \sum\limits_{(i,j): r(i,j)=1} (\theta^{jT} x^i - y^{i,j})^2 \\ + \frac{\lambda}{2} \sum\limits_{i=0}^{n_m} \sum\limits_{k=0}^n (x^i_k)^2 + \frac{\lambda}{2} \sum\limits_{j=0}^{n_u} \sum\limits_{k=0}^n (\theta^j_k)^2) \\
x^i_k := x^i_k - \alpha (\sum\limits_{j: r(i,j) = 1} (\theta^{j T} x^i - y^{i,j})\theta^j_k + \lambda x^i_k); \\
\theta^j_k := \theta^j_k - \alpha (\sum\limits_{i: r(i,j) = 1} (\theta^{j T} x^i - y^{i,j}) x^i_k + \lambda \theta^j_k); j = 0 \rightarrow \lambda = 0
$$

In [ ]:
# TODO: Implementar a função de custo e formação por descida de gradiente para filtros colaborativos
    
def cost_function_collaborative_filtering_regularized(x, theta, y, r, lambda_=0.):
    # DICA: Pode seguir estos passos:
    # Cuidado com as dimensões, escolher a ordem adequada e se transpõe ou não
    # Calcular a hipótese/previsão multiplicando X e Theta
    # A esse valor, resta Y
    # Para apenas formar sobre os valores conhecidos, multiplicar o resultado por R elemento a elemento
    # Cuidado, a multiplicação elemento a elemento é uma função de Numpy diferente à multiplicação vetorial
    # Elevar cada elemento ao quadrado
    # Somar o resultado final
    j = [...]
    
    # Calcular o fator de regularização para X
    # Elevar cada elemento de X ao quadrado
    # Somar todos os elementos na matriz 2D a um escalar
    x_reg = [...]
    # Calcular o fator de regularização para Theta
    # Recordar não regularizar a primeira coluna
    # Elevar ao quadrado todos os elementos restantes
    # Somar a um escalar
    theta_reg = [...]
    
    return 1/2 * (j + lambda_ * (x_reg + theta_reg))

def gradient_descent_collaborative_filtering_regularized(x, theta, y, r, lambda_=0., alpha=1e-3, n_iter=1e3, e n_iter = int(n_iter) 
    # Converter n_iter a int para poder usar em range()
                                                         
    # Inicializar j_hist com o historial de valores da função de custo
    j_hist = []
    # Adicionar como primeiro valor de custo o custo da função de custo para os valores iniciais
    j_hist.append(cost_function_collaborative_filtering_regularized([...]))
                                                         
    for l in range(n_iter):
        # Inicializar uma theta y x para atualizar com o gradiente com arrays do mesmo tamanho dos originais
        # e valores de vetor vazio (mais otimizado), zeros ou aleatórios
        theta_grad = [...]
        x_grad = [...]
                                                         
        for j in range(n_u):
            # Calcular o gradiente para atualizar theta nesta iteração
            # DICA: Pode seguir estes passos
            # Multiplicar X pela fila j de theta transposta (não theta_grad)
            # Resta a coluna j de Y
            # Multiplicar o resultado pela fila j de R
            # Multiplicar o resultado por X, cuidado com as dimensões e transposições se necessário
            # Somar o resultado final para obter uma fila, com cuidado sobre o eixo que se realiza a soma
            theta_grad[j,:] = [...]
                                                         
            # Para toda theta_grad exceto a primeira coluna, adicionar o termo de regularização
            # lambda * (fila de) theta
            if [...]:
                theta_grad[j,:] += [...]
                                                         

        for i in range(m):
            # Calcular o gradiente para atualizar X nesta iteração
            # Seguir passos semelhantes ao gradiente de theta para implementar a função correspondente
            # Somar o termo de regularização lambda * x
            x_grad[i,:] = [...]
                                                         
        # Atualizar X e Theta restando alpha * gradientes
        x -= alpha * x_grad
        theta -= alpha * theta_grad
                                                         
        # Se necessitar, comprovar como se vão atualizando X e Theta
        #print('\nValores de X e Theta atualizados:')
        #print(x)
        #print(theta)
                                                         
        # Calcular o custo nesta iteração e adicionando ao historial de custos 
        j_cost = cost_function_collaborative_filtering_regularized([...]) j_hist.append(j_cost)
                                                         
        # Se não é a primeira iteração e a diferença absoluta entre o custo e o da iteração anterior
        # menor que e, declara a convergência
        if [...]:
            print('Converge em iteração nº', l)
                                                         
            break
    else:
        print('Nº máx. de iterações {} alcançado'.format(n_iter))
                                                         
    return j_hist, x, theta

# Formação do modelo

Uma vez implementadas as funções correspondentes, vamos formar o modelo.

Para isso, completar a seguinte célula de código com passos equivalentes a outros modelos de exercícios prévios

In [ ]:
# TODO: Formar um modelo de sistema de recomendação por filtros colaborativos

# Gerar uma X e Theta inicial com valores aleatórios e o mesmo tamanho que X_verd e Theta_verd
x_init = [...]
theta_init = [...]

alpha = 1e-2 
lambda_ = 0. 
e = 1e-3 
n_iter = 1e4
print('Hiper-parâmetros usados:')
print('Alpha:', alpha, 'Lambda:', lambda_, 'Error:', e, 'Nº iter', n_iter)

t0 = time.time()
j_hist, x, theta = gradient_descent_collaborative_filtering_regularized(x_init, theta_init, y, r, lambda_, alp 
print('Duração do formação:', time.time() - t0)

                                                                        
print('\nÚltimos 10 valores da função de custo:') 
print(j_hist[-10:])
print('\nError mínimo:') 
print(min(j_hist))

Como temos feito em ocasiões prévias, representar graficamente a evolução da função de custo para comprovar que a formação do modelo foi correto:

In [ ]:
# TODO: Representar graficamente a função de custo da formação do modelo vs el nº de iterações
plt.figure()

# Representar o historial da função de custo
plt.plot([...])

# Adicionar um título, etiquetas em ambos os eixos do gráfico e uma grade
[...]

plt.show()

# Realização de previsões de recomendações

Uma vez formado o modelo, podemos resolver a matriz de recomendações *Y*, que contém como comentávamos tanto as valorizações emitidas pelos utilizadores como uma previsão da valorização de cada utilizador para cada exemplo.

Recordar que utilizávamos a matriz *R* para marcar com um “1” as valorizações reais e com um “0” aquelas que foram previstas e não eram conhecidas de antemão.

Para realizar uma previsão e recomendar exemplos aos utilizadores (p. ex. películas), seguir as instruções para completar a seguinte célula de código:

In [ ]:
# TODO: Realizar previsões de exemplos para os utilizadores

# Mostrar as valorizações da matriz Y
print('Valorizações de Y:')
print(y[:10,:10] * r[:10, :10]) # Limita o nº de filas e colunas de Y se necessário
# No resultado, um valor de "0." indica um “0.” nessa posição em R, ou que essa valorização inicial não é con

# Calcular as previsões obtidas pelo modelo a partir de X e Theta
# Recordar: Y = X * Theta^T
y_pred = [...]

print('\nPrevisões de valorizações:') 
print(y_pred[:10,:10])

# Calcular os resíduos das previsões
# Recordar que os resíduos são a diferença em valores absolutos entre o valor real conhecido e as previsões
# Recordar calcular apenas quando a valorização inicial é conhecida, multiplicando os resíduos por R
y_residuo = [...]

print('\nResíduos do modelo:') 
print(y_residuo[:10,:10])

# Mostrar as previsões e valorizações iniciais de um utilizador dado
jj = 1 # Escolher o n.º de utilizador ou índice entre 0 e n_u que quer

print('\nValorizações reais e previstas para o utilizador nº {}:'.format(jj + 1)) 
print(y_pred[:,jj])

# Ordenar os índices dos exemplos que recomendaríamos a cada utilizador em função das suas valorizações
# Recordar eliminar da lista as valorizações emitidas inicialmente pelo utilizador
# Para as eliminar, pode multiplicar as previsões por (r[:,jj] == 0.)
# Para extrair os índices ordenados de um array pode utilizar np.argsort()
# Para reordenar os índices de maior a menor pode utilizar np.flip() 
print('\nValorizações previstas para o utilizador nº {}:'.format(jj + 1)) print([...])

y_pred_ord = [...]

print('\nÍndices dos exemplos a recomendar para o utilizador {}, em função das valorizações previstas:'.f print(y_pred_ord)

# Voltar a formar incorporando novas valorizações

Para voltar a formar este modelo incorporando novas valorizações dos utilizadores, apenas há que modificar a *Y* inicial com as novas valorizações e marcar com um “1” a posição na matriz *R*.

Seguir as instruções da seguinte célula para incorporar 2 novas valorizações

In [ ]:
# TODO: Incorporar 2 novas valorizações de utilizadores a um exemplo à sua escolha

# Escolher um índice de utilizador e de exemplo
i_1 = 2
j_1 = 2
i_2 = 3
j_3 = 3

# Escolher uma valorização. Habitualmente tomam valores entre [0, 2)
y[...] = 1.
y[...] = 1.

# Marca as novas valorizações em R
r[...] = 1.
r[...] = 1.

Agora voltar a formar o modelo reexecutando a célula de formação do modelo e as células seguintes até à célula anterior. 

Comprovar como agora essas posições mostram a nova valorização e não uma previsão do modelo.

# Encontrar exemplos e utilizadores similares

Para encontrar a semelhança entre 2 elementos, podemos computar a distância euclídea entre ambos.

A distância euclídea neste espaço n-dimensional representará a diferença acumulada entre os coeficientes desses elementos, igual que uma distância num plano 2D ou 3D é a diferença acumulada entre as coordenadas desses pontos.

Encontrar exemplos e utilizadores similares seguindo as instruções da seguinte célula:

In [ ]:
# TODO: Encontrar exemplos e utilizadores similares entre si

# Calcular a semelhança entre os 4 primeiros exemplos (X)
dist_ej = distance.cdist([...])

print('Semelhança entre os 4 primeiros exemplos:') 
print(dist_ej)

# Calcular a semelhança entre os 4 primeiros utilizadores (Theta)
dist_us = distance.cdist([...])

print('Semelhança entre os 4 primeiros utilizadores:') 
print(dist_us)

# Calcular o exemplo mais semelhante ao primeiro
i_ej_similar = [...] 
ej_similar = [...]

print('Coeficientes do exemplo nº {} para os 5 primeiros utilizadores:'.format(0 + 1)) 
print(x[0,:5])
print('O exemplo mais semelhante ao nº {} é o exemplo nº {}'.format(0 + 1, i_ej_similar)) 
print('Coeficientes do exemplo nº {} para os 5 primeiros utilizadores:'.format(i_ej_similar)) 
print(ej_similar[:5])

# Calcular o utilizador mais semelhante ao primeiro
i_us_similar = [...] 
us_similar = [...]

print('Coeficientes do utilizador nº {} para os 5 primeiros exemplos:'.format(0 + 1)) 
print(theta[0,:5])
print('O utilizador mais semelhante ao nº {} é o utilizador nº {}'.format(0 + 1, i_us_similar)) 
print('Coeficientes do utilizador nº {} para os 5 primeiros exemplos:'.format(i_us_similar)) 
print(us_similar[:5])

## Bónus: Comprovar que sucede se não temos um n.º mínimo de valorizações

*O que sucede se não temos um nº mínimo de valorizações? E se há algum exemplo que não conta com nenhuma valorização de nenhum utilizador ou um utilizador que não valorizou nenhum exemplo?*

*Acreditar que, nesse caso, poderíamos formar o modelo e obter resultados para esses exemplos e utilizadores?*

Para o comprovar, pode p. ex. diminuir a percentagem de valorizações iniciais até um valor baixo, p. ex. um 25%, e comprovar que sucede com a evolução da função de custo da formação